In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [16]:
# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\HONOR\.cache\kagglehub\datasets\uciml\sms-spam-collection-dataset\versions\1


In [17]:

data = pd.read_csv('spam.csv', encoding='ISO-8859-1')
data = data[['v1', 'v2']]
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


In [18]:
#Беру из датасета первые 5560 строк, чтобы обучить модель (данные будут использоваться для обучения 
# и тестирования 75% на 25%), остальные строки датасета оставлю для наглядного тестирования

# Выделяем первые 5560 строк для обучения и тестирования модели
train_test_data = data.iloc[:5560]

In [19]:
ros = RandomOverSampler(random_state=42)

X = train_test_data['v2'].values.reshape(-1, 1)
y = train_test_data['v1']                        

X_resampled, y_resampled = ros.fit_resample(X, y)

balanced_data = pd.DataFrame(X_resampled, columns=['v2'])
balanced_data['v1'] = y_resampled

print("Class counts after oversampling:\n", balanced_data['v1'].value_counts())

Class counts after oversampling:
 v1
ham     4815
spam    4815
Name: count, dtype: int64


In [20]:
train_test_data['v1'] = train_test_data['v1'].astype(str)

train_test_data['v1'] = train_test_data['v1'].map({'ham': 0, 'spam': 1})

train_test_data = train_test_data.dropna(subset=['v1', 'v2'])

X_text = train_test_data['v2'].values.tolist()
y = train_test_data['v1'].values

C:\Users\HONOR\AppData\Local\Temp\ipykernel_2484\2693287613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_data['v1'] = train_test_data['v1'].astype(str)
C:\Users\HONOR\AppData\Local\Temp\ipykernel_2484\2693287613.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_data['v1'] = train_test_data['v1'].map({'ham': 0, 'spam': 1})


In [21]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

max_length = 200
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_padded)
X_test_scaled = scaler.transform(X_test_padded)

model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]


print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.9280575539568345
F1-Score: 0.6491228070175439
ROC-AUC: 0.7413086889256919


### Чтобы протестировать модель возьму столбцы датасета, которые не были использованы в обучении модели. Выделенные строки относятся к SPAM.

1. No. I meant the calculation is the same. That  &lt;#&gt; units at  &lt;#&gt; . This school is really expensive. Have you started practicing your accent.
2. Because its important. And have you decided if you are doing 4years of dental school or if you'll just do the nmde exam.
3. "Sorry, I'll call later"
4. if you aren't here in the next  &lt;#&gt;  hours imma flip my shit
5. Anything lor. Juz both of us lor.
6. Get me out of this dump heap. My mom decided to come to lowes. BORING.
7. Ok lor... Sony ericsson salesman... I ask shuhui then she say quite gd 2 use so i considering...
8. Ard 6 like dat lor.
9. Why don't you wait 'til at least wednesday to see if you get your .
10. Huh y lei...

**11. "REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"**


**12. "This is the 2nd time we have tried 2 contact u. U have won the �750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate."**

13. Will �_ b going to esplanade fr home?
14. "Pity, * was in mood for that. So...any other suggestions?"
15. The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free
16. Rofl. Its true to its name




In [27]:

new_data = [
    "No. I meant the calculation is the same. That <#> units at <#>. This school is really expensive. Have you started practicing your accent. Because it's important. And have you decided if you are doing 4 years of dental school or if you'll just do the nmde exam.",
    "Because its important. And have you decided if you are doing 4years of dental school or if you'll just do the nmde exam.",
    "Sorry, I'll call later",
    "if you aren't here in the next <#> hours imma flip my shit",
    "Anything lor. Juz both of us lor.",
    "Get me out of this dump heap. My mom decided to come to lowes. BORING.",
    "Ok lor... Sony ericsson salesman... I ask shuhui then she say quite gd 2 use so i considering...",
    "Ard 6 like dat lor.",
    "Why don't you wait 'til at least wednesday to see if you get your .",
    "Huh y lei...",
    "REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode",
    "This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. To claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.",
    "Will �_ b going to esplanade fr home?",
    "'Pity, * was in mood for that. So...any other suggestions?'",
    "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
    "Rofl. Its true to its name"
]

# Преобразуем текстовые данные в числовые последовательности с помощью обученного Tokenizer
new_sequences = tokenizer.texts_to_sequences(new_data)
new_padded = pad_sequences(new_sequences, maxlen=200, padding='post')

# Масштабируем новые данные с помощью обученного StandardScaler
new_scaled = scaler.transform(new_padded)

# Делаем предсказания с использованием обученной модели
y_manual = model.predict(new_scaled)

# Преобразуем метки в читаемый формат, если необходимо
results = pd.DataFrame({'text': new_data, 'label': y_manual})

# Печатаем результаты
results

,text,label
0,No. I meant the calculation is the same. That ...,0
1,Because its important. And have you decided if...,0
2,"Sorry, I'll call later",0
3,if you aren't here in the next <#> hours imma ...,0
4,Anything lor. Juz both of us lor.,0
5,Get me out of this dump heap. My mom decided t...,0
6,Ok lor... Sony ericsson salesman... I ask shuh...,0
7,Ard 6 like dat lor.,0
8,Why don't you wait 'til at least wednesday to ...,0
9,Huh y lei...,0
